# Aufgabe 1: PID Controller

In der folgenden Übung sollen sie einem Roboter-Auto einen _PID-Controller_ hinzufügen, wie er in der Vorlesung vorgestellt wurde.

## a)

Erläutern Sie zunächst die Bedeutung des Akronyms PID. Gehen Sie dabei auch kurz auf die einzelnen Elemente ein.

Erläutern Sie im Anschluss möglichst kurz und präzise den Begriff _Windup_ und wie er in diesem Kontext entstehen könnte.

> **Hinweis:** Kommen Sie nach der Bearbeitung der nächsten Teilaufgabe inkl. der Gewichtungen auf die Erläuterung des Begriffes Windup zurück.

### Antwort
_Fügen Sie hier ihren Text ein_

## Das Roboter Auto

Damit wir dieses Simulieren können laden wir zunächst einige externe Bibliotheken:

In [ ]:
# This is just for jupyter notebook
%matplotlib widget

# Adding dependencies
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon, Arrow
from matplotlib.collections import PatchCollection
from ipywidgets import Button, VBox

In [ ]:
class RobotCar:
    '''Creates a RobotCar and the corresponding plotting logic to draw the route graph

    Feel free to look at the code, but you should not need to change anything here
    The task is to implement the control function in the next cell

    Note: for a slightly better overview the official PEP8 style guide is partially ignored
    '''

    def __init__(self, control_fct, initial_pose=[0., 4., 0.], command=[0.1, 0.]):
        '''just the constructor'''
        self.pose       = np.array(initial_pose)  # Robot's pose [x, y, theta]
        self.command    = np.array(command)       # Robot's command [velocity, turning angle]
        self.hist_poses = np.array([[0.], [4.]])  # History of robot's positions for plotting
        # The function to control the car - usually reacts to an error
        self.control_function = control_fct
        # Create a figure and axis
        figure, axis  = plt.subplots(1, 1, figsize=(8, 4))
        self.fig = figure
        self.ax  = axis
        # Set the y-axis limits and aspect ratio
        self.ax.set_ylim(-6, 6)
        self.ax.set_aspect('equal')
        # Initialize the error values
        self.error = 0.
        # Continue with drawing the car
        self.init_visualization_patches()

        # Initialize the noise level - see task 3
        self.noise_level = 0.
        self.systematic_steering_bias = 0.
        self.rng = np.random.default_rng(seed=42)


    def set_plot_length(self, length=23):
        '''Sets the length of the plot'''
        self.ax.set_xlim(-1, length)                            # Set the x-axis limits
        self.ax.plot([-2., length +2], [0.,0.], color="green")  # Draw the base plot

        
    def init_visualization_patches(self):
        '''Creates the robot (body and wheels) and the direction arrow'''
        body_shape = [[-1, -0.5], [-1, 0.5], [1, 0.5], [1, -0.5]]
        body = Polygon(body_shape, closed=True, fc='r', ec='gray')
        left_front_wheel  = Rectangle( (0.3,  0.5), 0.4, 0.1, fc='gray', ec='black', lw=3)
        left_rear_wheel   = Rectangle((-0.7,  0.5), 0.4, 0.1, fc='gray', ec='black', lw=3)
        right_front_wheel = Rectangle( (0.3, -0.6), 0.4, 0.1, fc='gray', ec='black', lw=3)
        right_rear_wheel  = Rectangle((-0.7, -0.6), 0.4, 0.1, fc='gray', ec='black', lw=3)
        vehicle = [body, left_front_wheel, left_rear_wheel, right_front_wheel, right_rear_wheel]
        # Assemble!
        self.robot = PatchCollection(vehicle, match_original=True)
        self.arrow_left = Arrow(1,0,1,0, width=0.5, color="blue")
        # Add the patches to the axis
        self.ax.add_collection( self.robot )
        self.arr_hook = self.ax.add_patch( self.arrow_left )

        (self.line_robot_hist,) = self.ax.plot(self.hist_poses[0], self.hist_poses[1], zorder=0)

        
    def update_visualization(self):
        '''Updates the visualization of the robot's pose and history'''
        r = mpl.transforms.Affine2D().rotate(self.pose[2])
        t = mpl.transforms.Affine2D().translate(self.pose[0], self.pose[1])
        tra = r + t + self.ax.transData
        self.robot.set_transform(tra)
        self.line_robot_hist.set_data(self.hist_poses)
        self.arr_hook.remove()
        self.arrow_left = Arrow(1, 0, 1, self.command[1] * 15, width=0.5, color="blue")
        self.arr_hook = self.ax.add_patch( self.arrow_left )
        self.arrow_left.set_transform(tra)

        
    def reset_position(self, button=None):
        '''Resets the position of the robot'''
        self.pose = np.array([0., 4., 0.])
        self.hist_poses = np.array([[0.], [4.]])
        self.update_visualization()

        
    def simulate_one_step(self, button):
        '''Simulates one step of the robot's motion'''
        self.error = -self.pose[1] + self.noise_level * self.rng.normal()
        self.command[1] = self.control_function( self ) + self.systematic_steering_bias
        # Limit the turning angle
        max_turning_angle = 0.03
        if self.command[1] < -max_turning_angle:
            self.command[1] = -max_turning_angle
        elif self.command[1] > max_turning_angle:
            self.command[1] = max_turning_angle 
        # Simulate one step of the robot's motion
        self.pose[0] += np.cos(self.pose[2]) * self.command[0]
        self.pose[1] += np.sin(self.pose[2]) * self.command[0]
        self.pose[2] += self.command[1]
        self.hist_poses = np.append(self.hist_poses, np.array([[self.pose[0]], [self.pose[1]]]), axis=1)

    
    def simulation_run_anim(self, button=None, steps=100):
        '''Run the simulation for a certain number of steps'''
        for _ in range(0, steps):
            self.simulate_one_step(button)
            self.update_visualization()
            self.fig.canvas.draw()


    def simulation_run(self, button=None):
        ''' Run the simulation for a certain number of steps '''
        for _ in range(0, 600):
            self.simulate_one_step(button)
        self.update_visualization()
        self.fig.canvas.draw()


## b)

Ersetzen Sie im folgenden Block die Steuerungsfunktion durch einen PID Controller. Signatur und Parameter bleiben dabei identisch.

Erarbeiten Sie hierbei:
- die zu setzenden Werte
- sauber getrennte Teilgleichungen (ganz im Sinne der Modularisierung)
- sinnvolle Gewichtungen für die Elemente
- tragen Sie sich als Autor ein

> **Hinweise:**
> - `robot_car.error` ist hierbei die Abweichung von der Ideallinie
> - mit Modularisierung sind in diesem Fall keine eigenen Funktionen, sondern gut lesbare Zuweisungen gemeint
> - Signatur und Parameter bedeutet die oberste Zeile: `def control_function( robot_car ):`

In [ ]:
# Example Solution
def control_function(robot_car):
    '''Not so simple control function that steers the robot towards the center'''
    __author__ = "Florian Bohlken"
    rc = robot_car

    # Check if values exist
    if not hasattr(rc, 'prev_error'):
        rc.prev_error = rc.error
    if not hasattr(rc, 'integrated_error'):
        rc.integrated_error = 0.

    # Set values for this iteration
    rc.integrated_error += rc.error

    # Weights
    K_p = 0.02
    K_d = 0.5
    K_i = 0.0001

    # Separate into proportional, derivative and integral parts
    proportional = K_p * rc.error
    derivative = K_d * (rc.error - rc.prev_error) / 2
    integral = K_i * rc.integrated_error

    # Calculate steering
    steering = proportional + derivative + integral

    # Set values for next iteration
    rc.prev_error = rc.error

    return steering

## Ausgabe

Hier können Sie zunächst die Beispielfunktion und folgend ihre Implementierungen des PID-Controllers testen.

In [ ]:
# Create a RobotCar instance
robot_car = RobotCar(control_function)

# TODO: (Optional) Set a parameter to change the plot length
robot_car.set_plot_length(40)

# Create buttons and sliders for simulation control
button_reset = Button(description="Reset")
button_step = Button(description="Step")
button_run_anim = Button(description="Run Animation")
button_run = Button(description="Run 400 steps")

# Define the event handlers for buttons and sliders
button_reset.on_click(robot_car.reset_position)
button_step.on_click(robot_car.simulate_one_step)
button_run_anim.on_click(robot_car.simulation_run_anim)
button_run.on_click(robot_car.simulation_run)

# Display the buttons and sliders in a VBox
controls = VBox([button_reset, button_step, button_run_anim, button_run])
display(controls)

# Start the simulation
robot_car.update_visualization()
plt.show()

## c)

Folgend werden dem obigen System einerseits ein Signalrauschen hinzugefügt, sowie andererseits ein systematischer Fehler.

Beschreiben Sie kurz und präzise:
- wodurch diese beiden Eigenschaften in einem System auftreten können
- die Auswirkungen, die eine Änderung der Werte jeweils erzeugt

> **Hinweise zur Steuerung:**
> - Diese erfolgt im Rahmen von -0.03 bis 0.03
>   - entsprechend sollte der Fehler auch ziemlich klein sein!
> - positive Werte steuern nach links (steuern nach "oben")

In [ ]:
robot_car.reset_position()

# Introduce noise and systematic error
# width of zero centered gaussian noise, the standard deviation
robot_car.noise_level = 0.1
# simply added onto the steering angle: Use 0.002 as an example
robot_car.systematic_steering_bias = 0.01

# Display the buttons and sliders in a VBox
controls = VBox([button_reset, button_step, button_run_anim, button_run])
display(controls)

# Start the simulation
robot_car.update_visualization()
plt.show()

### Beschreibung

_Fügen Sie hier Ihren Text ein_